In [1]:
# importing dependencies
from eeg_transformer import *
from train import *

from sklearn.model_selection import train_test_split
from scipy.signal import detrend, filtfilt, butter, iirnotch, welch
import numpy as np
import matplotlib.pyplot as plt
import random
import os


# torch.nn is a module that implements varios useful functions and functors to implement flexible and highly
# customized neural networks. We will use nn to define neural network modules, different kinds of layers and
# diffrent loss functions
import torch.nn as nn

# torch.nn.functional implements a large variety of activation functions and functional forms of different
# neural network layers. Here we will use it for activation functions.
import torch.nn.functional as f

# torch is the Linear Algebra / Neural Networks library
import torch
from torch.utils.data import Dataset, DataLoader

# Setting Global Determinism
SEED = 0

def set_seeds(seed=SEED):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    torch.manual_seed
    np.random.seed(seed)

def set_global_determinism(seed=SEED):
    set_seeds(seed=seed)

    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
    torch.use_deterministic_algorithms(True)


set_global_determinism(seed=SEED)

c:\Users\Willian Oliveira\anaconda3\envs\pytorch_env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Loading dataset
ds = np.load('char_dataset.npz')
X = ds['x']
Y = ds['y']
print(X.shape)
print(Y.shape)

(230, 14, 1280)
(230,)


In [3]:
# Sampling data into 250ms duration
def sampling250ms(X, Y):
    X_new = np.zeros((36110,32,14))
    Y_new = np.zeros((36110,))
    npt = 32
    stride = 8
    ctr = 0
    for i in range(0, X.shape[0]):
        y = Y[i]
        a= X[i,:,:]
        a = a.transpose()
        val = 0
        while val<=(len(a)-npt):
            x = a[val:val+npt,:]
            X_new[ctr,:,:] = x
            Y_new[ctr] = y
            val = val+stride
            ctr = ctr+1
    return X_new, Y_new

In [4]:
X_new, Y_new = sampling250ms(X, Y)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_new, Y_new, test_size=0.2, random_state=SEED)

print(f'''
X_train shape:{X_train.shape} -> Train Labels: {y_train.shape[0]}
X_test shape:{X_test.shape} -> Test Labels: {y_test.shape[0]}''')


X_train shape:(28888, 32, 14) -> Train Labels: 28888
X_test shape:(7222, 32, 14) -> Test Labels: 7222


In [6]:
class dataset(Dataset):
    def __init__(self, X, y, train=True):
        self.X = X
        self.y = y
        self.train=train

In [7]:
def get_loaders(train_X, train_y, test_X, test_y):
    train_set, test_set = dataset(train_X, train_y, True), dataset(test_X, test_y, False)
    data_loader_train = torch.utils.data.DataLoader(
        train_set, 
        batch_size=1, 
        num_workers=1,
        pin_memory=True, 
        drop_last=False,
    )
    data_loader_test = torch.utils.data.DataLoader(
            test_set, 
            batch_size=1, 
            num_workers=1,
            pin_memory=True, 
            drop_last=False,
    )
    dataloaders = {
        'train': data_loader_train,
        'test': data_loader_test
    }
    return dataloaders

In [8]:
loaders = get_loaders(X_train, y_train, X_test, y_test)
dataloader_train = loaders['train']
dataloader_test = loaders['test']

In [9]:
opt = {}
opt['Transformer-layers'] = 2
opt['Model-dimensions'] = 256
opt['feedford-size'] = 512
opt['headers'] = 8
opt['dropout'] = 0.1
opt['src_d'] = 14 # input dimension
opt['tgt_d'] = 128 # output dimension

In [10]:
criterion = nn.MSELoss() # mean squared error

# setup model using hyperparameters defined above
model = make_model(opt['src_d'],opt['tgt_d'],opt['Transformer-layers'],opt['Model-dimensions'],opt['feedford-size'],opt['headers'],opt['dropout'])

# setup optimization function
model_opt = NoamOpt(model_size=opt['Model-dimensions'], factor=1, warmup=400,
        optimizer = torch.optim.Adam(model.parameters(), lr=0.015, betas=(0.9, 0.98), eps=1e-9))
total_epoch = 10
train_losses = np.zeros(total_epoch)
test_losses = np.zeros(total_epoch)

for epoch in range(total_epoch):
    model.train()
    train_loss = run_epoch(data_gen(dataloader_train), model, 
              SimpleLossCompute(model.generator, criterion, model_opt))
    train_losses[epoch]=train_loss

    if (epoch+1)%10 == 0:
        torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': model_opt.optimizer.state_dict(),
                    'loss': train_loss,
                    }, 'model_checkpoint/'+str(epoch)+'.pth')            
        torch.save(model, 'model_save/model%d.pth'%(epoch)) # save the model

    model.eval() # test the model

    test_loss = run_epoch(data_gen(dataloader_test), model, 
            SimpleLossCompute(model.generator, criterion, None))

    test_losses[epoch] = test_loss
    print('Epoch[{}/{}], train_loss: {:.6f},test_loss: {:.6f}'
              .format(epoch+1, total_epoch, train_loss, test_loss))